# Extra Examples - Merging

Heres a dataset dumped directly from a database, so we need to stitch it together ourselves.
https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings

The dataset comes with a README file that outlines where everything comes from which might help.

Lets try to:

1. Merge all restaurant data
2. Merge all user data
3. Merge restaurant data and user data together using user ratings
4. Realise that we've merged too much, and merge user ratings + user profile + geoplaces
5. Use some groupby power and determine the top five restaurants in the dataset

In [1]:
import pandas as pd
import os

files = [f for f in os.listdir() if f.endswith(".csv")]
print(files)

['chefmozparking.csv', 'chefmozaccepts.csv', 'userpayment.csv', 'geoplaces2.csv', 'rating_final.csv', 'usercuisine.csv', 'chefmozcuisine.csv', 'chefmozhours4.csv', 'userprofile.csv']


## Merging restaurant data

In [54]:
restaurant_df = None
restaurant_files = []
user_files = []
for f in files:
    if 'user' in f:
        user_files.append(f)
    elif f == 'rating_final.csv':
        pass
    else:
        restaurant_files.append(f)
def df_zip(frame, lst, merge_on):
    for i in lst:
        if frame is None:
            frame = pd.read_csv(i)
        else:
            frame = frame.merge(pd.read_csv(i), on = merge_on)
    return frame

In [56]:
restaurant_df = df_zip(restaurant_df, restaurant_files, 'placeID')

## Merging User data

In [58]:
user_df = None
user_df = df_zip(user_df, user_files, 'userID')

## Merging User ratings as well

In [59]:
ratings_df = pd.read_csv("rating_final.csv")
full_df = ratings_df.merge(user_df, on="userID")
full_df = full_df.merge(restaurant_df, on="placeID")

In [61]:
full_df[0:4]

,userID,placeID,rating,food_rating,service_rating,Upayment,Rcuisine_x,latitude_x,longitude_x,smoker,...,accessibility,price,url,Rambience,franchise,area,other_services,Rcuisine_y,hours,days
0,U1077,135085,2,2,2,VISA,Mexican,22.156469,-100.98554,false,...,no_accessibility,medium,?,familiar,f,closed,none,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;
1,U1077,135085,2,2,2,VISA,Mexican,22.156469,-100.98554,false,...,no_accessibility,medium,?,familiar,f,closed,none,Fast_Food,00:00-00:00;,Sat;
2,U1077,135085,2,2,2,VISA,Mexican,22.156469,-100.98554,false,...,no_accessibility,medium,?,familiar,f,closed,none,Fast_Food,00:00-00:00;,Sun;
3,U1077,135085,2,2,2,cash,Mexican,22.156469,-100.98554,false,...,no_accessibility,medium,?,familiar,f,closed,none,Fast_Food,00:00-00:00;,Mon;Tue;Wed;Thu;Fri;


## Merge Subsets

In [65]:
geo_df = pd.read_csv('geoplaces2.csv')
user_prof_df = pd.read_csv('userprofile.csv')
target_df = ratings_df.merge(geo_df, on='placeID')
target_df = target_df.merge(user_prof_df, on='userID')

## Top 5 restaurants based off rating

Note to answer this we didn't actually need the user profile data. But we might use it to remove votes from users that don't satisfy criteria (for example, we might want to make sure the user has been to multiple restaurants, or is a certain age, or doesnt have suspicious voting trends - aka giving everyone a one).

In [89]:
rankings = target_df.groupby('name').agg(rating_mean=("rating", "mean"),
                                          rating_count=("rating", 'nunique')).reset_index()
#     {"Sales": [("SalesMean", "mean"), ("SalesUncert", mc_uncert)], "Customers": "count"})


In [96]:
#  Work to be done here regarding the rating count of course.

rankings.sort_values(by=['rating_mean'], ascending=False)[:5]

,name,rating_mean,rating_count
55,Restaurant Las Mananitas,2.000000,1
44,Michiko Restaurant Japones,2.000000,1
108,emilianos,2.000000,1
99,cafe punta del cielo,1.833333,2
32,La Estrella de Dimas,1.800000,2
